In [17]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import lightgbm as lgb
import safetensors
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer, 
    TrainingArguments,
    DataCollatorWithPadding
)
from datasets import Dataset
from tqdm import tqdm
from itertools import permutations
import warnings
warnings.filterwarnings('ignore')

In [18]:
def create_label_mappings():
    """24가지 순열에 대한 라벨 매핑 생성"""
    all_permutations = list(permutations([0, 1, 2, 3]))
    perm_to_label = {perm: idx for idx, perm in enumerate(all_permutations)}
    label_to_perm = {idx: perm for idx, perm in enumerate(all_permutations)}
    
    print(f"총 {len(all_permutations)}개의 순열 클래스 생성")
    print("예시 매핑:")
    for i in range(5):
        print(f"  라벨 {i}: {all_permutations[i]}")
    
    return perm_to_label, label_to_perm

# 순열 매핑 생성
perm_to_label, label_to_perm = create_label_mappings()

총 24개의 순열 클래스 생성
예시 매핑:
  라벨 0: (0, 1, 2, 3)
  라벨 1: (0, 1, 3, 2)
  라벨 2: (0, 2, 1, 3)
  라벨 3: (0, 2, 3, 1)
  라벨 4: (0, 3, 1, 2)


In [19]:
def prepare_roberta_data(train_df, perm_to_label):
    """RoBERTa용 데이터 준비"""
    processed_data = []
    
    for _, row in train_df.iterrows():
        sentences = [row[f"sentence_{i}"] for i in range(4)]
        answer_tuple = tuple([row[f"answer_{i}"] for i in range(4)])
        text = " [SEP] ".join(sentences)
        label = perm_to_label[answer_tuple]
        
        processed_data.append({
            "text": text,
            "label": label,
            "original_sentences": sentences,
            "answer": answer_tuple
        })
    
    return processed_data

def augment_roberta_data_advanced(train_df, perm_to_label, multiplier=4):
    """고급 데이터 증강"""
    augmented_data = []
    
    # 원본 데이터
    original_data = prepare_roberta_data(train_df, perm_to_label)
    augmented_data.extend(original_data)
    
    # 다양한 증강 방법
    for aug_round in range(multiplier - 1):
        for _, row in train_df.iterrows():
            sentences = [row[f"sentence_{i}"] for i in range(4)]
            original_answer = [row[f"answer_{i}"] for i in range(4)]
            
            if aug_round == 0:
                # 랜덤 셔플
                indices = list(range(4))
                np.random.shuffle(indices)
            elif aug_round == 1:
                # 순환 이동
                shift = np.random.randint(1, 4)
                indices = [(i + shift) % 4 for i in range(4)]
            else:
                # 부분 교환
                indices = list(range(4))
                i, j = np.random.choice(4, 2, replace=False)
                indices[i], indices[j] = indices[j], indices[i]
            
            shuffled_sentences = [sentences[i] for i in indices]
            new_answer = tuple([indices.index(original_answer[i]) for i in range(4)])
            
            text = " [SEP] ".join(shuffled_sentences)
            label = perm_to_label[new_answer]
            
            augmented_data.append({
                "text": text,
                "label": label,
                "original_sentences": shuffled_sentences,
                "answer": new_answer
            })
    
    print(f"고급 데이터 증강 완료: {len(original_data)} → {len(augmented_data)}")
    return augmented_data

In [20]:
print("데이터 로드 중...")
train_df = pd.read_csv('./train.csv')
print(f"원본 학습 데이터: {len(train_df)}개")

print("\n고급 데이터 증강 중...")
augmented_data = augment_roberta_data_advanced(train_df, perm_to_label, multiplier=4)

print("\n학습/검증 데이터 분할...")
train_data, valid_data = train_test_split(
    augmented_data, 
    test_size=0.2, 
    random_state=42,
    stratify=[item["label"] for item in augmented_data]
)

print(f"학습 데이터: {len(train_data)}개")
print(f"검증 데이터: {len(valid_data)}개")

데이터 로드 중...
원본 학습 데이터: 7351개

고급 데이터 증강 중...
고급 데이터 증강 완료: 7351 → 29404

학습/검증 데이터 분할...
학습 데이터: 23523개
검증 데이터: 5881개


In [21]:
# =============================================================================
# 셀 5: Stacking 앙상블 클래스 정의 (완전 수정됨)
# =============================================================================
class AdvancedStackingEnsemble:
    """최고 성능 Stacking 앙상블"""
    
    def __init__(self, n_folds=5, random_state=42):
        self.n_folds = n_folds
        self.random_state = random_state
        self.base_models = []
        self.meta_model = None
        self.base_model_configs = [
            {
                'name': 'bert_small', 
                'model_name': 'klue/bert-base',  # BERT 아키텍처 (첫 번째)
                'learning_rate': 1.5e-5,
                'epochs': 25,
                'batch_size': 48,
                'warmup_steps': 200
            },
            {
                'name': 'roberta_small',
                'model_name': 'klue/roberta-small',
                'learning_rate': 2e-5,
                'epochs': 20,
                'batch_size': 64,
                'warmup_steps': 150
            },
            {
                'name': 'electra_small',
                'model_name': 'monologg/koelectra-small-v3-discriminator',  # ELECTRA 아키텍처 (마지막)
                'learning_rate': 3e-5,
                'epochs': 15,
                'batch_size': 80,
                'warmup_steps': 100
            }
        ]
        
    def train_base_model(self, config, train_dataset, valid_dataset, tokenizer, device):
        """개별 베이스 모델 학습"""
        print(f"\n🔥 {config['name']} 모델 학습 시작...")
        
        # 모델 생성
        model = AutoModelForSequenceClassification.from_pretrained(
            config['model_name'],
            num_labels=24,
            cache_dir='C:/huggingface_cache'
        )
        model.to(device)
        
        # 학습 설정 (조기 종료 포함)
        training_args = TrainingArguments(
            output_dir=f"./results_{config['name']}",
            learning_rate=config['learning_rate'],
            per_device_train_batch_size=config['batch_size'],
            per_device_eval_batch_size=128,
            gradient_accumulation_steps=2,
            num_train_epochs=config['epochs'],
            eval_strategy="steps",
            eval_steps=150,
            save_strategy="steps", 
            save_steps=150,
            save_total_limit=2,
            load_best_model_at_end=True,
            metric_for_best_model="accuracy",
            greater_is_better=True,
            fp16=True,
            dataloader_pin_memory=True,
            dataloader_num_workers=6,
            warmup_steps=config['warmup_steps'],
            weight_decay=0.01,
            logging_steps=20,
            report_to=None,
        )
        
        # 트레이너 생성 (조기 종료 콜백 포함)
        from transformers import EarlyStoppingCallback
        
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=valid_dataset,
            tokenizer=tokenizer,
            data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
            compute_metrics=lambda eval_pred: {
                "accuracy": accuracy_score(eval_pred.label_ids, np.argmax(eval_pred.predictions, axis=1))
            },
            callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
        )
        
        # 학습 실행
        trainer.train()
        
        # 모델 저장
        model_save_path = f"./results_{config['name']}/final"
        trainer.save_model(model_save_path)
        
        print(f"✅ {config['name']} 모델 학습 완료!")
        
        return {
            'model': model,
            'trainer': trainer,
            'config': config,
            'save_path': model_save_path
        }
    
    def generate_meta_features(self, models, data, device):
        """메타 특성 생성 (K-Fold 교차 검증)"""
        print("\n🧠 메타 특성 생성 중...")
        
        # 데이터를 DataFrame으로 변환
        df = pd.DataFrame(data)
        
        # K-Fold 설정
        kf = KFold(n_splits=self.n_folds, shuffle=True, random_state=self.random_state)
        
        # 메타 특성 저장할 배열
        meta_features = np.zeros((len(data), len(models) * 24))  # 각 모델당 24개 클래스 확률
        
        for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
            print(f"  Fold {fold + 1}/{self.n_folds} 처리 중...")
            
            # 검증 데이터 추출
            val_data = df.iloc[val_idx]
            
            # 각 베이스 모델로 예측
            for model_idx, model_info in enumerate(models):
                model = model_info['model']
                model.eval()
                
                # 해당 모델의 토크나이저 로드
                tokenizer = AutoTokenizer.from_pretrained(
                    model_info['config']['model_name'],
                    cache_dir='C:/huggingface_cache'
                )
                
                fold_predictions = []
                
                # 배치 단위로 예측
                for _, row in val_data.iterrows():
                    text = row['text']
                    
                    inputs = tokenizer(
                        text,
                        return_tensors="pt",
                        truncation=True,
                        padding=True,
                        max_length=512
                    ).to(device)
                    
                    with torch.no_grad():
                        outputs = model(**inputs)
                        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
                        fold_predictions.append(probabilities.cpu().numpy()[0])
                
                # 메타 특성에 저장
                start_col = model_idx * 24
                end_col = (model_idx + 1) * 24
                meta_features[val_idx, start_col:end_col] = np.array(fold_predictions)
        
        return meta_features
    
    def train_meta_model(self, meta_features, labels):
        """메타 모델 학습"""
        print("\n🎯 메타 모델 학습 중...")
        
        # LightGBM을 메타 모델로 사용 (가장 효과적)
        self.meta_model = lgb.LGBMClassifier(
            objective='multiclass',
            num_class=24,
            boosting_type='gbdt',
            num_leaves=31,
            learning_rate=0.05,
            feature_fraction=0.9,
            bagging_fraction=0.8,
            bagging_freq=5,
            verbose=0,
            random_state=self.random_state
        )
        
        # 메타 모델 학습
        self.meta_model.fit(meta_features, labels)
        
        print("✅ 메타 모델 학습 완료!")
        
        return self.meta_model
    
    def fit(self, train_data, valid_data, device):
        """전체 스택킹 앙상블 학습"""
        print("🚀 Stacking 앙상블 학습 시작!")
        
        # 데이터셋 생성
        train_df = pd.DataFrame(train_data)
        valid_df = pd.DataFrame(valid_data)
        
        train_dataset = Dataset.from_pandas(train_df)
        valid_dataset = Dataset.from_pandas(valid_df)
        
        # 1단계: 베이스 모델들 학습
        print("\n📚 1단계: 베이스 모델들 학습")
        for config in self.base_model_configs:
            # 각 모델에 맞는 토크나이저 로드
            tokenizer = AutoTokenizer.from_pretrained(
                config['model_name'],
                cache_dir='C:/huggingface_cache'
            )
            print(f"✅ {config['name']} 토크나이저 로드 완료")
            
            # 토크나이징 함수
            def tokenize_function(examples):
                return tokenizer(
                    examples["text"],
                    truncation=True,
                    padding=True,
                    max_length=512
                )
            
            # 각 모델별로 데이터셋 토크나이징
            current_train_dataset = train_dataset.map(tokenize_function, batched=True)
            current_valid_dataset = valid_dataset.map(tokenize_function, batched=True)
            
            current_train_dataset = current_train_dataset.remove_columns(["text", "original_sentences", "answer"])
            current_valid_dataset = current_valid_dataset.remove_columns(["text", "original_sentences", "answer"])
            
            # 모델 학습
            model_info = self.train_base_model(config, current_train_dataset, current_valid_dataset, tokenizer, device)
            self.base_models.append(model_info)
        
        # 2단계: 메타 특성 생성
        print("\n🔧 2단계: 메타 특성 생성")
        meta_features = self.generate_meta_features(self.base_models, train_data, device)
        labels = [item['label'] for item in train_data]
        
        # 3단계: 메타 모델 학습
        print("\n🎓 3단계: 메타 모델 학습")
        self.train_meta_model(meta_features, labels)
        
        print("\n🎉 Stacking 앙상블 학습 완료!")
        
        return self
    
    def predict(self, test_data, device):
        """스택킹 앙상블 예측"""
        print("\n🔮 Stacking 앙상블 예측 중...")
        
        # 테스트 데이터로 베이스 모델 예측
        test_meta_features = []
        
        for model_info in self.base_models:
            model = model_info['model']
            model.eval()
            
            # 해당 모델의 토크나이저 로드
            tokenizer = AutoTokenizer.from_pretrained(
                model_info['config']['model_name'],
                cache_dir='C:/huggingface_cache'
            )
            
            model_predictions = []
            
            for text in tqdm(test_data, desc=f"{model_info['config']['name']} 예측"):
                inputs = tokenizer(
                    text,
                    return_tensors="pt",
                    truncation=True,
                    padding=True,
                    max_length=512
                ).to(device)
                
                with torch.no_grad():
                    outputs = model(**inputs)
                    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
                    model_predictions.append(probabilities.cpu().numpy()[0])
            
            test_meta_features.append(np.array(model_predictions))
        
        # 메타 특성 결합
        final_meta_features = np.hstack(test_meta_features)
        
        # 메타 모델로 최종 예측
        final_predictions = self.meta_model.predict(final_meta_features)
        final_probabilities = self.meta_model.predict_proba(final_meta_features)
        
        return final_predictions, final_probabilities

In [22]:
print("디바이스 설정 중...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"디바이스: {device}")

# GPU 메모리 사용량 체크 함수
def print_gpu_utilization():
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        usage_percent = (allocated / total) * 100
        print(f"GPU 메모리: {allocated:.2f}GB 사용 / {total:.2f}GB 전체 ({usage_percent:.1f}% 사용)")
    else:
        print("CUDA 사용 불가")

print_gpu_utilization()

디바이스 설정 중...
디바이스: cuda
GPU 메모리: 0.25GB 사용 / 8.00GB 전체 (3.2% 사용)


In [ ]:
# Stacking 앙상블 생성 및 학습
stacking_ensemble = AdvancedStackingEnsemble(n_folds=5, random_state=42)

# 학습 실행 (tokenizer 파라미터 제거)
stacking_ensemble.fit(train_data, valid_data, device)

🚀 Stacking 앙상블 학습 시작!

📚 1단계: 베이스 모델들 학습
✅ bert_small 토크나이저 로드 완료


Map: 100%|██████████| 5881/5881 [00:00<00:00, 8625.73 examples/s]



🔥 bert_small 모델 학습 시작...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy
150,3.109400,2.962562,0.151165
300,1.493800,1.331347,0.469988
450,1.032500,0.915630,0.584935
600,0.741500,0.674062,0.741200
750,0.548300,0.510139,0.805135
900,0.413800,0.409693,0.848495
1050,0.261300,0.338250,0.883013
1200,0.216300,0.247987,0.920762
1350,0.134700,0.207902,0.937936
1500,0.084700,0.154539,0.952389


✅ bert_small 모델 학습 완료!
✅ roberta_small 토크나이저 로드 완료


Map: 100%|██████████| 5881/5881 [00:00<00:00, 6158.13 examples/s]



🔥 roberta_small 모델 학습 시작...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy
150,3.181600,3.087374,0.088420
300,1.912500,1.761048,0.420507
450,1.175300,1.050144,0.606359
600,0.774600,0.742543,0.726747
750,0.590900,0.548103,0.807006
900,0.419000,0.423061,0.853256
1050,0.278900,0.319827,0.898487
1200,0.211600,0.243814,0.923482
1350,0.148700,0.206569,0.938106
1500,0.103000,0.164750,0.952049


In [ ]:
print("테스트 데이터 예측 중...")

# 테스트 데이터 로드
test_df = pd.read_csv("./test.csv")
print(f"테스트 데이터: {len(test_df)}개")

# 테스트 데이터 전처리
test_texts = []
for _, row in test_df.iterrows():
    sentences = [row[f"sentence_{i}"] for i in range(4)]
    text = " [SEP] ".join(sentences)
    test_texts.append(text)

# Stacking 앙상블로 예측 (tokenizer 파라미터 제거)
final_predictions, final_probabilities = stacking_ensemble.predict(test_texts, device)

# 예측 결과를 순열로 변환
predicted_orders = []
confidences = []

for i, pred_label in enumerate(final_predictions):
    predicted_order = list(label_to_perm[pred_label])
    confidence = np.max(final_probabilities[i])
    
    predicted_orders.append(predicted_order)
    confidences.append(confidence)

# 평균 신뢰도 출력
avg_confidence = np.mean(confidences)
print(f"\n평균 예측 신뢰도: {avg_confidence:.4f}")

테스트 데이터 예측 중...
테스트 데이터: 1780개

🔮 Stacking 앙상블 예측 중...


roberta_v3 예측: 100%|██████████| 1780/1780 [00:13<00:00, 133.28it/s]


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5

평균 예측 신뢰도: 0.9554


In [ ]:
print("Submission 파일 생성 중...")

# 샘플 제출 파일 로드
sample_submission = pd.read_csv("./sample_submission.csv")

# 예측 결과를 제출 형식으로 변환
for i in range(4):
    sample_submission[f"answer_{i}"] = [pred[i] for pred in predicted_orders]

# 파일 저장
submission_filename = "stacking_ensemble_submission.csv"
sample_submission.to_csv(submission_filename, index=False)

print(f"제출 파일 저장 완료: {submission_filename}")

# 최종 결과 요약
print(f"\n" + "="*60)
print(f"🏆 고급 Stacking 앙상블 완료!")
print(f"📊 베이스 모델 수: {len(stacking_ensemble.base_models)}개")
print(f"🎯 메타 모델: LightGBM")
print(f"🔮 평균 예측 신뢰도: {avg_confidence*100:.2f}%")
print(f"📁 제출 파일: {submission_filename}")
print(f"="*60)

# 개별 모델 성능도 확인해보기
print(f"\n개별 베이스 모델 정보:")
for i, model_info in enumerate(stacking_ensemble.base_models):
    config = model_info['config']
    print(f"  모델 {i+1}: {config['name']} (LR: {config['learning_rate']}, Epochs: {config['epochs']})")

Submission 파일 생성 중...
제출 파일 저장 완료: stacking_ensemble_submission.csv

🏆 고급 Stacking 앙상블 완료!
📊 베이스 모델 수: 3개
🎯 메타 모델: LightGBM
🔮 평균 예측 신뢰도: 95.54%
📁 제출 파일: stacking_ensemble_submission.csv

개별 베이스 모델 정보:
  모델 1: roberta_v1 (LR: 2e-05, Epochs: 20)
  모델 2: roberta_v2 (LR: 1.5e-05, Epochs: 25)
  모델 3: roberta_v3 (LR: 3e-05, Epochs: 15)
